In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
## Hyper-parameters

input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
## MNIST dataset (images and labels)

train_dataset = torchvision.datasets.MNIST(root = '../../data',
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True)

test_dataset = torchvision.datasets.MNIST(root = '../../data',
                                          train = False,
                                          transform = transforms.ToTensor())

0it [00:00, ?it/s]

9920512it [00:01, 8188012.06it/s]                            


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 135620.42it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2219764.74it/s]                            
0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 50806.52it/s]            


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
## Data loader (input pipeline)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

In [0]:
## Logistic regression model

model = nn.Linear(input_size, num_classes)

In [0]:
## Loss and optimizer
## nn.CrossEntropyLoss() computes softmax internally

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr = learning_rate)

In [7]:
## Train the model

total_step = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    ## Reshape images to (batch_size, input_size)
    images = images.reshape(-1, 28 * 28)
    
    ## Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    ## Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i + 1) % 100 == 0:
      print ('Epoch [{} / {}], Step [{} / {}], Loss: {:.4f}'
             .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1 / 5], Step [100 / 600], Loss: 2.1699
Epoch [1 / 5], Step [200 / 600], Loss: 2.0850
Epoch [1 / 5], Step [300 / 600], Loss: 2.0012
Epoch [1 / 5], Step [400 / 600], Loss: 1.9782
Epoch [1 / 5], Step [500 / 600], Loss: 1.8307
Epoch [1 / 5], Step [600 / 600], Loss: 1.8130
Epoch [2 / 5], Step [100 / 600], Loss: 1.6897
Epoch [2 / 5], Step [200 / 600], Loss: 1.7140
Epoch [2 / 5], Step [300 / 600], Loss: 1.5641
Epoch [2 / 5], Step [400 / 600], Loss: 1.5670
Epoch [2 / 5], Step [500 / 600], Loss: 1.4779
Epoch [2 / 5], Step [600 / 600], Loss: 1.4182
Epoch [3 / 5], Step [100 / 600], Loss: 1.4747
Epoch [3 / 5], Step [200 / 600], Loss: 1.3140
Epoch [3 / 5], Step [300 / 600], Loss: 1.2854
Epoch [3 / 5], Step [400 / 600], Loss: 1.2919
Epoch [3 / 5], Step [500 / 600], Loss: 1.1712
Epoch [3 / 5], Step [600 / 600], Loss: 1.3322
Epoch [4 / 5], Step [100 / 600], Loss: 1.2823
Epoch [4 / 5], Step [200 / 600], Loss: 1.1967
Epoch [4 / 5], Step [300 / 600], Loss: 1.2976
Epoch [4 / 5], Step [400 / 600], L

In [8]:
## Test the model
## In test phase, we don't need to compute gradients (for memory efficiency)

with torch.no_grad():
  
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.reshape(-1, 28 * 28)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
  print('Accuracy of the model on the 10000 test images: {} %'
        .format(100 * correct / total))

Accuracy of the model on the 10000 test images: 82 %


In [0]:
## Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')